In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


# Part 1 Data mining

In [ ]:

driver = webdriver.Edge()
driver.get("https://www.daft.ie/property-for-sale/ireland")
result = driver.find_element(By.CLASS_NAME,"styles__SearchH1-sc-1t5gb6v-3")
num = result.text.split()[0]
num = num.replace(",","")
num =2000
count =0
all_links = []
while(count<= int(num)):
    driver.get("https://www.daft.ie/property-for-sale/ireland?from="+str(count)+"&pageSize=20")
    a_tags = driver.find_elements(By.XPATH,"//ul[@class='SearchPagestyled__SearchResults-v8jvjf-3 hzMJok']/li/a")
    all_links.extend([a_tag.get_attribute('href') for a_tag in a_tags])
    count += 20


filename = "links.txt"
print(filename)

with open(filename, 'wb') as f:
    print("Saving to", filename)
    for link in all_links:
        print(link)
        string_link = str(link)+ "\n"
        f.write(string_link.encode('utf-8'))
    f.close()

driver.quit()


# Part 2 Data Mining

In [ ]:


with open('links.txt', 'r') as f:
    urls = f.readlines()
    f.close()
    
def get_element(soup, key, value, tag,default='N/A'):
    try:
        if tag == 'ul':
            data = soup.find(tag,{key:value}).find('li')
        else:
            data = soup.find(tag,{key:value})
            
        if data:
            return data.text.strip()
        return default
    except:
        return default
    
def get_elements(soup, key, value, tag,default='N/A',index_data=1):
    try:
        datas = soup.find_all(tag,{key:value})
        if len(datas)>0:
            if(index_data == 0):
                for i in range(len(datas)):
                    datas[i] = datas[i].text.strip()
                return datas
            else:
                return datas[index_data-1].text.strip()
        return default
    except:
        return default 

data = []
driver = webdriver.Edge()
for url in urls:
    url = url.replace("\n","")
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    address = get_element(soup,'data-testid','address','h1')
    price = get_element(soup,'class','TitleBlock__StyledCustomHeading-sc-1avkvav-5 blbeVq','h2')
    beds = get_element(soup,'data-testid','beds','p')
    baths = get_element(soup,'data-testid','baths','p')
    floor_area = get_element(soup,'data-testid','floor-area','p')
    property_type = get_element(soup,'data-testid','property-type','p')
    selling_method = get_element(soup,'class','styles__InfoSection-sc-15fxapi-7 ikMOXo','ul')
    condition = get_element(soup,'class','Statistics__StyledValue-sc-15tgae4-2 MCfSO','p')
    date_statistics = get_elements(soup,'class','Statistics__StyledLabel-sc-15tgae4-1 iDjRee','p',index_data=1)
    viewer = get_elements(soup,'class','Statistics__StyledLabel-sc-15tgae4-1 iDjRee','p',index_data=2)
    property_features_or_facilities = get_elements(soup,'class','PropertyDetailsList__PropertyDetailsListItem-sc-1cjwtjz-1 hqJwsU','li')
    data.append({'address':address,'price':price,'beds':beds,'baths':baths,'floor_area':floor_area,'property_type':property_type,'selling_method':selling_method,'condition':condition,'date_statistics':date_statistics,'viewer':viewer,'property_features_or_facilities':property_features_or_facilities})
    print("Count: ",len(data))




filename = "data.csv"
fieldnames = ['address','price','beds','baths','floor_area','property_type','selling_method','condition','date_statistics','viewer','property_features_or_facilities']
with open(filename,'w',newline='',encoding='utf-8') as f:
    print("Saving to ",filename)
    csv_writer = csv.DictWriter(f,fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(data)
    f.close()
    
driver.quit()

Data will be scrabbed from the website on obtaining the property links and write it into links.txt file. Then load each of the property link to get the detailed information and write it into the data.csv file